In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_ibm_runtime import (QiskitRuntimeService,Session,EstimatorV2,EstimatorOptions)


In [ ]:
def apply_alice_measurement(qc, basis: int, alice_qubit=0):
    if basis == 1:
        qc.ry(-np.pi/4, alice_qubit)
    elif basis == 2:
        pass
    else:
        qc.h(alice_qubit)

def apply_bob_measurement(qc, basis: int, bob_qubit=1):
    if basis == 1:
        pass
    elif basis == 2:
        qc.ry(np.pi/4, bob_qubit)
    else:
        qc.ry(-np.pi/4, bob_qubit)
def prepare_singlet_state():
    qc = QuantumCircuit(2)
    qc.x(0)
    qc.h(0)
    qc.cx(0, 1)
    qc.z(1)
    return qc

def build_e91_circuits(n_pairs, alice_bases, bob_bases):
    circuits = []
    for i in range(n_pairs):
        qc = prepare_singlet_state()
        apply_alice_measurement(qc, alice_bases[i], alice_qubit=0)
        apply_bob_measurement(qc, bob_bases[i], bob_qubit=1)
        circuits.append(qc)
    return circuits

def run_e91_local(n_pairs=100, shots=1024):
    alice_bases = np.random.randint(0, 3, size=n_pairs)
    bob_bases   = np.random.randint(0, 3, size=n_pairs)
    circuits = build_e91_circuits(n_pairs, alice_bases, bob_bases)
    observables = [Pauli("ZZ")] * n_pairs
    estimator = AerEstimator()
    job = estimator.run(circuits, observables, shots=shots)
    result = job.result()
    corr_values = result.values 
    ab_dict = {}
    for (a, b, val) in zip(alice_bases, bob_bases, corr_values):
        ab_dict.setdefault((a,b), []).append(val)
    avg_E = {k: np.mean(v) for k, v in ab_dict.items()}
    e_xw = avg_E.get((0,0), 0)  # (X, W)
    e_xv = avg_E.get((0,2), 0)  # (X, V)
    e_zw = avg_E.get((2,0), 0)  # (Z, W)
    e_zv = avg_E.get((2,2), 0)  # (Z, V)
    s_value = e_xw - e_xv + e_zw + e_zv
    alice_key = []
    bob_key   = []
    for (a_basis, b_basis, cval) in zip(alice_bases, bob_bases, corr_values):
    
        if (a_basis == 1 and b_basis == 0) or (a_basis == 2 and b_basis == 1):
            if np.random.rand() < p_same:
                # same => (0,0) or (1,1)
                bit = np.random.randint(2)
                alice_key.append(bit)
                # Bob's anticorrelated => 1 - bit
                bob_key.append(1 - bit)
            else:
                # different => (0,1) or (1,0)
                bit = np.random.randint(2)
                alice_key.append(bit)
                bob_key.append(bit)
            
    bob_key = [1 - b for b in bob_key]
    # compute QBER
    mismatches = sum(a != b for a,b in zip(alice_key, bob_key))
    qber = mismatches / len(alice_key) if len(alice_key) > 0 else None

    print("\n[E91 Local Simulation — Corrected Logic]")
    print(f"Number of pairs: {n_pairs}, Shots: {shots}")
    print(f"CHSH S = {s_value:.3f}   (max = ±2√2 ≈ 2.828...)")
    print(f"QBER   = {qber}")
    print(f"Key length = {len(alice_key)}")

    return {
        "S": s_value,
        "QBER": qber,
        "alice_key": alice_key,
        "bob_key": bob_key,
        "corr_values": corr_values,
        "avg_E": avg_E
    }


# -------------------------------------------------------------------------
# Simple test invocation
if __name__ == "__main__":
    results = run_e91_local(n_pairs=100, shots=1024)
    print("S-value:", results["S"])
    print("QBER:", results["QBER"])
    print("Alice key:", results["alice_key"])
    print("Bob   key:", results["bob_key"])


C:\Users\utkar\AppData\Local\Temp\ipykernel_7116\2812903349.py:125: DeprecationWarning: Estimator has been deprecated as of Aer 0.15, please use EstimatorV2 instead.
  results = run_e91_local(n_pairs=100, shots=1024)
C:\Users\utkar\AppData\Local\Temp\ipykernel_7116\2812903349.py:125: DeprecationWarning: Option approximation=False is deprecated as of qiskit-aer 0.13. It will be removed no earlier than 3 months after the release date. Instead, use BackendEstimator from qiskit.primitives.
  results = run_e91_local(n_pairs=100, shots=1024)



[E91 Local Simulation — Corrected Logic]
Number of pairs: 100, Shots: 1024
CHSH S = 2.822   (max = ±2√2 ≈ 2.828...)
QBER   = 0.0
Key length = 19
S-value: 2.822265625
QBER: 0.0
Alice key: [1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1]
Bob   key: [1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1]


In [3]:
service = QiskitRuntimeService(channel="ibm_quantum",  # ibm_cloud 
                               token = '8a3e35ba3172edac10b0179786cb911eb5368c636c1c4542d158e3d17dabb33b75b7c7069d4fa4919f3bbb758622444d139f28356c32f337f18397b08559b99d')
QiskitRuntimeService.save_account(channel='ibm_quantum', overwrite=True,
                                  token = '8a3e35ba3172edac10b0179786cb911eb5368c636c1c4542d158e3d17dabb33b75b7c7069d4fa4919f3bbb758622444d139f28356c32f337f18397b08559b99d')
backend = service.backend(name = "ibm_brisbane")
backend.num_qubits

127

In [ ]:
def run_e91_hardware_corrected(n_pairs=100, backend_name="ibm_brisbane"):
    # Generate random bases
    alice_bases = np.random.randint(0, 3, size=n_pairs)
    bob_bases = np.random.randint(0, 3, size=n_pairs)

    # Build and transpile circuits
    circuits = build_e91_circuits(n_pairs, alice_bases, bob_bases)
    pass_manager = generate_preset_pass_manager(optimization_level=1, backend=backend)
    transpiled_circuits = [pass_manager.run(circ) for circ in circuits]

    observables = []
    for qc in transpiled_circuits:
        q0 = qc.find_bit(qc.qubits[0]).index
        q1 = qc.find_bit(qc.qubits[1]).index
        label = ["I"] * qc.num_qubits
        label[q0] = "Z"
        label[q1] = "Z"
        observables.append(SparsePauliOp(Pauli("".join(label))))

    options = EstimatorOptions(
        resilience_level=1,
        dynamical_decoupling={"enable": True, "sequence_type": "XY4"}
    )

    with Session(backend=backend) as session:
        estimator = EstimatorV2(options=options)
        job = estimator.run(list(zip(transpiled_circuits, observables)))
        result = job.result()

    corr_values = [float(res.data.evs) for res in result]

    ab_dict = {}
    for (a, b, val) in zip(alice_bases, bob_bases, corr_values):
        ab_dict.setdefault((a, b), []).append(val)
    avg_E = {k: np.mean(v) for k, v in ab_dict.items()}

    e_xw = avg_E.get((0, 0), 0)
    e_xv = avg_E.get((0, 2), 0)
    e_zw = avg_E.get((2, 0), 0)
    e_zv = avg_E.get((2, 2), 0)
    s_value = e_xw - e_xv + e_zw + e_zv

    alice_key, bob_key = [], []
    for (a_basis, b_basis, cval) in zip(alice_bases, bob_bases, corr_values):
        if (a_basis == 1 and b_basis == 0) or (a_basis == 2 and b_basis == 1):
            p_same = 0.5 * (1 + cval)
            if np.random.rand() < p_same:
                bit = np.random.randint(2)
                alice_key.append(bit)
                bob_key.append(1 - bit)
            else:
                bit = np.random.randint(2)
                alice_key.append(bit)
                bob_key.append(bit)

    bob_key = [1 - b for b in bob_key]
    mismatches = sum(a != b for a, b in zip(alice_key, bob_key))
    qber = mismatches / len(alice_key) if len(alice_key) > 0 else None

    # Output
    print("\n[✔️ E91 QKD on Real IBM Quantum Hardware — Updated Logic]")
    print(f"Backend      : {backend.name}")
    print(f"CHSH S value : {s_value:.3f}")
    print(f"QBER         : {qber}")
    print(f"Key length   : {len(alice_key)}\n")

    return {
        "backend": backend.name,
        "S": s_value,
        "QBER": qber,
        "alice_key": alice_key,
        "bob_key": bob_key,
        "avg_E": avg_E
    }


In [7]:
if __name__ == "__main__":
    result = run_e91_hardware_corrected(n_pairs=127, backend_name="ibm_brisbane")

c:\Users\utkar\anaconda3\envs\myenv\lib\site-packages\qiskit_ibm_runtime\qiskit_runtime_service.py:857: UserWarning: Your current pending jobs are estimated to consume 893.4194092030581 quantum seconds, but you only have 600 quantum seconds left in your monthly quota; therefore, it is likely this job will be canceled
  warnings.warn(warning_message)



[✔️ E91 QKD on Real IBM Quantum Hardware — Updated Logic]
Backend      : ibm_brisbane
CHSH S value : 1.933
QBER         : 0.0
Key length   : 31

